In [ ]:
#Bryan Bennett
# 3/2/2021

In [ ]:
from sagemaker import get_execution_role
import pandas as pd
import sagemaker
import boto3
import os

In [ ]:
role = get_execution_role()
sess = sagemaker.Session()

In [ ]:
bucketName = 'signet-ring-cell-bryan'
folderName = 'challenge2-colorectal-dataset/tissue-train-pos-v1/'

In [ ]:
s3_client = boto3.client('s3')
bucket = s3_client.list_objects(Bucket=bucketName, Prefix=folderName)

In [ ]:
len(bucket['Contents'])

In [ ]:
# Bucket names for training and validation data

prefix = 'challenge2-colorectal-dataset/tumor_segmentation_model'

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'
train_annotation_channel = prefix + '/train_annotation'
validation_annotation_channel = prefix + '/validation_annotation'


s3_train_data = 's3://{}/{}'.format(bucketName, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucketName, validation_channel)
s3_train_annotation = 's3://{}/{}'.format(bucketName, train_annotation_channel)
s3_validation_annotation = 's3://{}/{}'.format(bucketName, validation_annotation_channel)

In [ ]:
#FIXME: Split train and validation sets here?
import shutil

#os.makedirs('data/train', exist_ok=True)
#os.makedirs('data/validation', exist_ok=True)
#os.makedirs('data/train_annotation', exist_ok=True)
#os.makedirs('data/validation_annotation', exist_ok=True)

#Create annotation file for each image
for img_num in range(len(bucket['Contents'])):
     if img_num%2==0:
            file_name = bucket['Contents'][img_num]['Key']
            with fs.open('{}/{}'.format(bucketName, file_name)) as f:
                data = f.read()
             #xml_to_json(data, bucketName, SRC_folderName, file_name)
            #place object

In [ ]:
# Set output location where artifact will be stored
s3_output_location = 's3://{}/{}/output'.format(bucketName, 'challenge2-colorectal-dataset/tumor_segmentation_model')

In [ ]:
# Get training uri

#Arguments in-order
training_image = sagemaker.image_uris.retrieve(region=sess.boto_region_name, framework='semantic-segmentation')
print (training_image)

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'semantic-segmentation', repo_version="latest")
print (training_image)